In [1]:
import pandas as pd
import numpy as np

In [2]:
df_p = pd.read_csv('data/finalized/proficiency.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,16,17,22,25,40,42,43,44,49,50,51,52,53,54,55,57,58,59,60,61,62,65,66,74,75,77,78,79,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,99,100,101,102,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# filter to relevant columns
df_p = df_p[['state', 'edchamp_name', 'year', 'district_id', 'school_id', 'grade', 'group', 'subject', 'st_quin', 'city_quin', 'st_quin_imp', 'city_quin_imp', 'grantid', 'grantid_grant_school', 'leagrantstrategy', 'schoolgrantstrategy_grant_school', 'startdate', 'startdate_grant_school','levelofedchampinvolvement', 'levelofedchampinvolvement_grant_school','supportorganizationsinvolved', 'supportorganizationsinvolved_grant_school','granttype', 'granttype_grant_school', 'grade_config']]
print(df_p.head(5))

  state edchamp_name  year district_id school_id grade         group subject  \
0    ca   Educate 78  2015       10017    130401   All  All Students     ELA   
1    ca   Educate 78  2015       10017    130419   All  All Students     ELA   
2    ca   Educate 78  2015       10017    118489   All  All Students     ELA   
3    ca   Educate 78  2015       10017    123968   All  All Students     ELA   
4    ca   Educate 78  2015       10017    123968   All  All Students     ELA   

   st_quin  city_quin  ...                   schoolgrantstrategy_grant_school  \
0      1.0        1.0  ...                                                NaN   
1      1.0        1.0  ...                                                NaN   
2      3.0        3.0  ...                                                NaN   
3      2.0        2.0  ...  Expansion (new school, existing operator); Imp...   
4      2.0        2.0  ...    Improvement (existing school, current operator)   

   startdate  startdate_grant_sc

In [4]:
print(df_p.dtypes)

state                                         object
edchamp_name                                  object
year                                           int64
district_id                                   object
school_id                                     object
grade                                         object
group                                         object
subject                                       object
st_quin                                      float64
city_quin                                    float64
st_quin_imp                                  float64
city_quin_imp                                float64
grantid                                      float64
grantid_grant_school                         float64
leagrantstrategy                              object
schoolgrantstrategy_grant_school              object
startdate                                     object
startdate_grant_school                        object
levelofedchampinvolvement                     

In [6]:
# filter to only grantees
print(df_p.shape)
df_p = df_p.dropna(subset=['grantid', 'grantid_grant_school'], how='all')
print(df_p.shape)

(7329136, 25)
(1428319, 25)


In [15]:
# change startdates to dates
df_p['startdate'] = df_p['startdate'].str.strip()
df_p['startdate_grant_school'] = df_p['startdate_grant_school'].str.strip()
df_p['startdate_dt'] = pd.to_datetime(df_p['startdate'], errors='coerce', infer_datetime_format=True)
df_p['startdate_grant_school_dt'] = pd.to_datetime(df_p['startdate_grant_school'], errors='coerce', infer_datetime_format=True)

del df_p['startdate']
del df_p['startdate_grant_school']

In [18]:
# find months and spring years
df_p['month_startdate'] = df_p.startdate_dt.dt.month
df_p['month_startdate_grant_school'] = df_p.startdate_grant_school_dt.dt.month
df_p['year_startdate'] = df_p.startdate_dt.dt.year
df_p['year_startdate_grant_school'] = df_p.startdate_grant_school_dt.dt.year

def calcSY(row):
    if row['month_startdate'] >= 8:
        return row['year_startdate'] + 1
    else:
        return row['year_startdate']

df_p['startdate_sy'] = df_p.apply(lambda x: calcSY(x), axis = 1)

def calcSYSchool(row):
    if row['month_startdate_grant_school'] >= 8:
        return row['year_startdate_grant_school'] + 1
    else:
        return row['year_startdate_grant_school']

df_p['startdate_sy_grant_school'] = df_p.apply(lambda x: calcSYSchool(x), axis = 1)

del df_p['month_startdate']
del df_p['month_startdate_grant_school']
del df_p['year_startdate']
del df_p['year_startdate_grant_school']

In [20]:
# calculate earliest grant
def calcEarliestGrant(row):
    if row['startdate_dt'] == np.datetime64('NaT'):
        return row['startdate_sy_grant_school']
    else:
        if row['startdate_grant_school_dt'] == np.datetime64('NaT'):
            return row['startdate_sy']
        else:
            if row['startdate_dt'] < row['startdate_grant_school_dt']:
                return row['startdate_sy']
            else:
                return row['startdate_sy_grant_school']

df_p['earliest_grant_sy'] = df_p.apply(lambda x: calcEarliestGrant(x), axis=1)

In [ ]:
# calculate min grant sy for every school and join back to school year

In [22]:
# calculate grant year
df_p['grant_year'] = df_p['year'] - df_p['earliest_grant_sy'] + 1
df_p.to_csv('temp.csv')

In [ ]:
# create grant year 0, and all following grant years
df_0 = df_p[df_p.grant_year.eq(0)]
df_5 = df_p[df_p.grant_year.eq(5)]
# del df_p

df_5 = df_5[['state', 'edchamp_name', 'year', 'district_id', 'school_id', 'grade', 'group', 'subject', 'st_quin', 'city_quin', 'st_quin_imp', 'city_quin_imp', 'grantid', 'grantid_grant_school', 'leagrantstrategy', 'schoolgrantstrategy_grant_school', 'startdate', 'startdate_grant_school','levelofedchampinvolvement', 'levelofedchampinvolvement_grant_school','supportorganizationsinvolved', 'supportorganizationsinvolved_grant_school','granttype', 'granttype_grant_school', 'grade_config']]

df = df_0.merge(df_5, on = ['state', 'edchamp_name', 'year', 'district_id', 'school_id', 'grade', 'group', 'subject'])